In [1]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import copy
import json
import numpy as np
import matplotlib.pyplot as plt 
import albumentations as A
from albumentations.pytorch import ToTensorV2
from models.efficientnet import EffNetModel
from pytorch_metric_learning import losses, miners, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from tqdm import tqdm
import wandb

In [2]:
from datasets_handlers.base_dataset import BaseImageDataset
from datasets_handlers.dataset_cub import cub200_dataset
from datasets_handlers.in_shop import in_shop_dataset

In [14]:
wandb.init(project="my-test-project", entity="ilya_fedorov")

wandb: Currently logged in as: ilya_fedorov (use `wandb login --relogin` to force relogin)


In [3]:
cub200_train = cub200_dataset(split='train')
cub200_test = cub200_dataset(split='test')

In [4]:
inshop_train = in_shop_dataset(split='train')
inshop_gallery = in_shop_dataset(split='gallery')
inshop_query = in_shop_dataset(split='query')

In [6]:
image_size = 224
train_transform = A.Compose(
    [
        #A.HorizontalFlip(p=0.5),
        #A.ImageCompression(quality_lower=99, quality_upper=100),
        #A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=10, border_mode=0, p=0.7),
        A.Resize(image_size, image_size),
        #A.CoarseDropout(max_height=int(image_size * 0.4), max_width=int(image_size * 0.4), max_holes=1, p=0.5),
        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
        ToTensorV2()
    ]
)

test_transform = A.Compose([
    A.Resize(image_size, image_size),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
    ToTensorV2()
])

In [7]:
dataset_train = BaseImageDataset(inshop_train, path_prefix='data/', augmentations=train_transform)
dataset_gallery = BaseImageDataset(inshop_gallery, path_prefix='data/', augmentations=test_transform)
dataset_query = BaseImageDataset(inshop_query, path_prefix='data/', augmentations=test_transform)

In [8]:
parameters = {
    'n_epochs': 10,
    'batch_size': 16,
    'lr': 1e-5
}

wandb.config = parameters

In [9]:
dataloader_train = DataLoader(
    dataset_train,
    batch_size=parameters['batch_size'],
    shuffle=True,
    num_workers=8,
    pin_memory=True
)

In [10]:
model = EffNetModel('efficientnet-b0').cuda()
model = model.train()

Loaded pretrained weights for efficientnet-b0


In [11]:
loss_func = losses.TripletMarginLoss()
#loss_func = losses.ArcFaceLoss(num_classes=200, embedding_size=1280).cuda()
#miner = miners.MultiSimilarityMiner()

In [12]:
optimizer = torch.optim.AdamW(model.parameters(), lr=parameters['lr'])
#opt2 = torch.optim.AdamW(loss_func.parameters(), lr=parameters['lr'])

In [15]:
for epoch in range(parameters['n_epochs']):
    for data, labels in tqdm(dataloader_train):
        data = data.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        embeddings = model(data)
        #hard_pairs = miner(embeddings, labels)
        loss = loss_func(embeddings, labels)
        wandb.log({"loss": loss})
        loss.backward()
        optimizer.step()
        #opt2.step()

100%|███████████████████████████████████████| 1618/1618 [01:16<00:00, 21.02it/s]


In [18]:
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)


### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    print("Computing accuracy")
    accuracies = accuracy_calculator.get_accuracy(
        test_embeddings, train_embeddings, test_labels, train_labels, False
    )
    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))

In [19]:
accuracy_calculator = AccuracyCalculator(include=("precision_at_1",), k=1)

In [20]:
test(dataset_gallery, dataset_query, model, accuracy_calculator)

100%|█████████████████████████████████████████| 445/445 [00:13<00:00, 33.51it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.3639752426501618


In [21]:
test(dataset_train, dataset_train, model, accuracy_calculator)

100%|█████████████████████████████████████████| 809/809 [00:23<00:00, 34.06it/s]


Computing accuracy
Test set accuracy (Precision@1) = 1.0
